In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [2]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [3]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json


In [4]:
# %%bash
# export GOOGLE_APPLICATION_CREDENTIALS="/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json"
# gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS


In [5]:
!gcloud auth list
# !gcloud config set account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [6]:
# in the command line. based on:
# https://nextflow.io/blog/2023/nextflow-with-gbatch.html
# gcloud services enable batch.googleapis.com compute.googleapis.com logging.googleapis.com
# gcloud iam service-accounts create eila-batch
# gcloud iam service-accounts list 




# gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/storage.admin"


In [7]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["run_seqspec_modify_atac"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [8]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [9]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [10]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/atac_barcode_and_subpool.sh

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/atac_barcode_and_subpool.sh


In [11]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/

atac_barcode_and_subpool.sh		 plot_insert_size_hist-Copy1.py
barcode_rank_functions.R		 plot_insert_size_hist.py
compute_tss_script.py			 run_tabix.sh
create_sample_filtered_fragment_file.sh  scrna_plot_atac_qc_metrics-Copy1.R
generate_barcode_metadata.sh		 scrna_plot_atac_qc_metrics.R


In [12]:

!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin/joint_cell_plotting.py
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin/joint_cell_plotting_density.R
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin/write_html.py






In [13]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/bin/


joint_cell_plotting.py	joint_cell_plotting_density.R  write_html.py


In [14]:
# !nextflow -self-update

In [15]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [16]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/

docker				   nf_data     nf_processes
hai-gcp-genomic-beb9207a40b8.json  nf_modules  nf_workflow


In [17]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint'

In [18]:
%%writefile nextflow_vm.config

manifest {
  description = 'A nexflow pipeline for fg single cell joint qc' 
  mainScript = 'nf_wf_fg_singlecell_joint.nf'
}

profiles {
  
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
    
}

// Define ENV_SYNAPSE_TOKEN as a parameter

params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_PY='joint_cell_plotting.py'
params.SC_JOINT_CELL_BARCODE_PLOT_RNA_METRICS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na'
params.SC_JOINT_CELL_BARCODE_PLOT_ATAC_METRICS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na1.na'
//Minimum number of UMIs/fragments required for a cell to be plotted
params.SC_JOINT_CELL_BARCODE_PLOT_REMOVE_LOW_YIELDING_CELLS=5
params.SC_JOINT_CELL_BARCODE_PLOT_METADATA_FILE='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na2.na'
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_UMIS=5
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_GENES=5
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_TSS=5
//Cutoff for minimum number of ATAC fragments
params.SC_JOINT_CELL_BARCODE_PLOT_MIN_FRAGS=5
params.SC_JOINT_CELL_BARCODE_PLOT_PKR='PKR123'
params.SC_JOINT_CELL_BARCODE_PLOT_SCRIPT_R='joint_cell_plotting_density.R'


params.SC_JOINT_CELL_BARCODE_PLOT_HTML_SCRIPT='write_html.py'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_ATAC_METRICS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_RNA_METRICS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na1.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_IMAGE_FILES='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na2.na'
params.SC_JOINT_CELL_BARCODE_PLOT_HTML_LOG_FILES='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na3.na'

    

Overwriting nextflow_vm.config


In [19]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint


In [20]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_joint/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/ATAC

In [21]:
# !rm .nextflow*

In [22]:
# !ls nextflow.config

In [23]:
!gcloud config list


[compute]
region = us-central1
zone = us-central1-a
[core]
account = eila-batch@hai-gcp-genomic.iam.gserviceaccount.com
disable_usage_reporting = True
project = hai-gcp-genomic

Your active configuration is: [default]


In [24]:
!nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_joint.nf` [zen_cray] DSL2 - revision: d926c7e73f
Dec 17, 2023 9:24:59 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
Before calling run_joint_barcode_cell_plotting
RNA Metrics: /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na
ATAC Metrics: /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na1.na
Uploading local `bin` scripts folder to gs://eila-nextflow-bucket/work/ATAC/tmp/c7/c556daa46b0a9c86f8b68927438246/bin
[-        ] process > run_joint_barcode_cell_plot... -
[-        ] process > run_joint_cell_plotting_den... -
[-        ] process > run_joint_metrics_html_report  -
After calling run_joint_barcode_cell_plotting
Before calling run_joint_cell_plotting_density
After calling run_joint_cell_plotting_density
Before run_joint_metrics_html_report
Before run_joint_metrics_html_report
After run_joint_metrics

In [25]:
!cat .nextflow.log

Dec-17 09:24:57.006 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_joint.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume
Dec-17 09:24:57.191 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Dec-17 09:24:57.225 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Dec-17 09:24:57.241 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Dec-17 09:24:57.242 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Dec-17 09:24:57.247 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Dec-17 09:24:57.263 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Dec-17 09:24:57.292 [main] DEBUG nextflow.config

In [26]:
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile google_batch_profile -resume

In [27]:
# !cat .nextflow.log 

In [28]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/seqspec/specs/dogmaseq-dig/fastqs

In [29]:
# !nextflow run /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [30]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [31]:
# !cat .command.out

In [32]:
# !nextflow run nf_wf_fg_singlecell.nf -c nextflow.config -with-docker eilalan/env_synapse:latest